In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import time

C:\ProgramData\Anaconda3\envs\Tensor2\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\envs\Tensor2\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\envs\Tensor2\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\envs\Tensor2\lib\site-packages\tensorflow\

### 1 加载数据集

In [2]:
mnist = input_data.read_data_sets("MNIST_data", one_hot=True)

W0819 14:56:20.321692  7696 deprecation.py:323] From <ipython-input-2-83231f068ae1>:1: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0819 14:56:20.322692  7696 deprecation.py:323] From C:\ProgramData\Anaconda3\envs\Tensor2\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0819 14:56:20.324692  7696 deprecation.py:323] From C:\ProgramData\Anaconda3\envs\Tensor2\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instruc

Extracting MNIST_data\train-images-idx3-ubyte.gz


W0819 14:56:20.548705  7696 deprecation.py:323] From C:\ProgramData\Anaconda3\envs\Tensor2\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0819 14:56:20.551705  7696 deprecation.py:323] From C:\ProgramData\Anaconda3\envs\Tensor2\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W0819 14:56:20.594708  7696 deprecation.py:323] From C:\ProgramData\Anaconda3\envs\Tensor2\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecat

Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


### 2 设定批次

In [3]:
batch_size = 50
n_batch = mnist.train.num_examples // batch_size  # 总共有多少个批次

with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, [None, 784], name='input_x')  # 每一行表示一个样本，内容是784为的像素值
    y = tf.placeholder(tf.float32, [None, 10], name='input_y')   # 每一行表示一个label，采用杜热编码

### 3 创建安神经网络（不含隐藏层）

In [4]:
w1 = tf.Variable(tf.random_normal([784, 100]) * 0.01)
b1 = tf.Variable(tf.random_normal([100]) * 0.01)
wx_plus_b_l1 = tf.matmul(x, w1) + b1
l1 = tf.nn.tanh(wx_plus_b_l1)
print(l1.shape)

w2 = tf.Variable(tf.random_normal([100,10]) * 0.01)
b2 = tf.Variable(tf.random_normal([10]) * 0.01)
wx_plus_b_l2 = tf.matmul(l1, w2) + b2
prediction = tf.nn.softmax(wx_plus_b_l2)


(?, 100)


### 4 优化器

In [5]:
# loss = tf.reduce_mean(tf.square(y-prediction))
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=prediction))
    tf.summary.scalar('loss', loss)
    
with tf.name_scope('train_step'):
    train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)

W0819 14:56:21.077735  7696 deprecation.py:323] From <ipython-input-5-f03ffb6d6254>:3: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



### 5 获取预测结果并计算准确率

In [6]:
with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.arg_max(prediction, 1), tf.arg_max(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar('accuracy', accuracy)

W0819 14:56:21.221744  7696 deprecation.py:323] From <ipython-input-6-b86b710cb5ec>:2: arg_max (from tensorflow.python.ops.gen_math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.math.argmax` instead


### 6 启动会话

In [7]:
merged = tf.summary.merge_all()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter('G:/tensorflow/logs2/',sess.graph)
    for epoch in range(51):
        time1 = time.time()
        for batch in range(n_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            _, summary = sess.run([train_step, merged], feed_dict={x:batch_x, y:batch_y})  
        writer.add_summary(summary, epoch)
        acc = sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels})  # 用测试集来预测
        print("第{0}圈  准确率是： {1}, 耗时: {2} 秒 ".format(epoch, acc, time.time()-time1))

第0圈  准确率是： 0.8702999949455261, 耗时: 0.962054967880249 秒 
第1圈  准确率是： 0.900600016117096, 耗时: 0.8650496006011963 秒 
第2圈  准确率是： 0.9103999733924866, 耗时: 0.8600492477416992 秒 
第3圈  准确率是： 0.9172999858856201, 耗时: 0.9530544281005859 秒 
第4圈  准确率是： 0.92330002784729, 耗时: 0.8830506801605225 秒 
第5圈  准确率是： 0.9265000224113464, 耗时: 0.8750500679016113 秒 
第6圈  准确率是： 0.9298999905586243, 耗时: 0.8700497150421143 秒 
第7圈  准确率是： 0.9320999979972839, 耗时: 0.8660495281219482 秒 
第8圈  准确率是： 0.9337000250816345, 耗时: 0.8830506801605225 秒 
第9圈  准确率是： 0.9362000226974487, 耗时: 1.0260586738586426 秒 
第10圈  准确率是： 0.9390000104904175, 耗时: 0.8760499954223633 秒 
第11圈  准确率是： 0.9412000179290771, 耗时: 0.8670496940612793 秒 
第12圈  准确率是： 0.9419000148773193, 耗时: 0.8660495281219482 秒 
第13圈  准确率是： 0.944100022315979, 耗时: 0.8620493412017822 秒 
第14圈  准确率是： 0.9451000094413757, 耗时: 0.8620491027832031 秒 
第15圈  准确率是： 0.9473999738693237, 耗时: 0.950054407119751 秒 
第16圈  准确率是： 0.9487000107765198, 耗时: 0.8710496425628662 秒 
第17圈  准确率是： 0.9492999911308289